論文<br>
https://cdn.openai.com/papers/dall-e-2<br>
<br>
GitHub<br>
https://github.com/LAION-AI/dalle2-laion<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/dalle2_laion_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

## ライブラリのインストール

In [ ]:
!pip install -q ipywidgets


# ライブラリのインポート

In [ ]:
%cd /content

import ipywidgets as widgets
import sys
import os
import json